In [7]:
########################## Información general ##########################
###Eliminación por Bloques a través de métodos de Jacobi y GaussSiedel###
##############1.- Repositorio de Funciones y Librerías###################

#################### Declaración de funciones generales##################
############# 2.- Métodos de Jacobi y Gauss_Seidel ######################

source('Jacobi_GaussSeidel.r')


[1] "Soluci\303\263n mediante metodo de Gauss-Seidel"
[1] "Matriz A:"
     [,1] [,2] [,3] [,4]
[1,]    0    3   -1    8
[2,]   -1   11   -1    3
[3,]    2   -1   10   -1
[4,]   10   -1    2    0
[1] "Vector b:"
[1]  15  25 -11   6
[1] "La matriz tiene alg\303\272n cero en la diagonal, comienza ordenamiento"
[1] "Matriz ordenada:"
     [,1] [,2] [,3] [,4]
[1,]   10   -1    2    0
[2,]   -1   11   -1    3
[3,]    2   -1   10   -1
[4,]    0    3   -1    8
[1] "Vector ordenado:"
[1]   6  25 -11  15
[1] "Iteracion 0"
[1] 0 0 0 0
[1] "Iteracion 1"
[1]  0.6000000  2.3272727 -0.9872727  0.8788636
[1] "nbr_Numerador: 2.32727272727273"
[1] "nbr_Denominador: 2.32727272727273"
[1] "nbr_Diff: 1"
[1] "Iteracion 2"
[1]  1.0301818  2.0369380 -1.0144562  0.9843412
[1] "nbr_Numerador: 0.430181818181818"
[1] "nbr_Denominador: 2.03693801652893"
[1] "nbr_Diff: 0.211190431270401"
[1] "Iteracion 3"
[1]  1.0065850  2.0035550 -1.0025274  0.9983509
[1] "nbr_Numerador: 0.0333829996243429"
[1] "nbr_Denominador: 2

In [8]:
########################## 3.- Eliminación por Bloques #########################################
# Este método consiste en eliminar un subconjunto de variables y resolver un sistema más pequeño.
# Si el sistema más pequeño asociado a la submatriz puede resolverse con las funciones anteriores
# entonces este método puede tener más eficiencia que uno que no trabaja por bloques.
  #
  # Parámetros
  # -------
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar dentro del método SEL
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método SEL ha convergido
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para resolver 
  #    los procedimientos que requieran SEL.
  #
  # Regresa
  # -------
  # vector
  #    c(x1,x2) como la solución del sistema

# Algoritmo

EliminacionBloques <- function(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo){
# Variables del algoritmo de eliminación por bloques
col.A <- ncol(mtrx_A)
lim <- col.A/2
A11 <- matrix(mtrx_A[1:lim,1:lim], nrow = lim, ncol=lim)
A12 <- matrix(mtrx_A[1:lim,(lim+1):col.A], nrow = lim, ncol=lim)
A21 <- matrix(mtrx_A[(lim+1):col.A,1:lim], nrow = lim, ncol=lim)
A22 <- matrix(mtrx_A[(lim+1):col.A,(lim+1):col.A], nrow = lim, ncol=lim)
b1<- c(vct_B[1:lim])
b2<- c(vct_B[(lim+1):col.A])
X01<- c(vct_X0[1:lim])
X02<- c(vct_X0[(lim+1):col.A])

#1) Calculamos y = invA11*b1 y Y = invA11*A12 a través de los métodos del bloque anterior donde 
#   los sistemas a evaluar en la forma Ax=b son A11*y=b1 y A11*Y=A12 respectivamente.

###Con el método de Jacobi o GaussSeidel
y <- funcResolverSE(A11, b1, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
#print("y:")
#print(y)
    
#No podemos resolver A11Y=A12 como tal cual con nuestro método iterativo, pues Y es matriz y el lado derecho (A12) también.
#Es necesario partirla en problemas donde la incógnita y el lado derecho sean vectores.
#Debemos resolver para cada columna de Y, la ecuación A11Yi=A12i , donde Yi es la columan i de Y y A12i es la columa i de A12.
#Así ya podemos aplicar a cada uno de estos SEL nuestro método Jacobi Gauss-Seidel, y al final juntar las soluciones en forma columnar para obtener la matriz Y.    

cols.A12 <- ncol(A12)
rows.A12 <- nrow(A12)

Ycol <- A12[,1]
Ysol <- funcResolverSE(A11, Ycol, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
Y <- Ysol

for (i in 2:cols.A12){
Ycol <- A12[,i]
Ysol <- funcResolverSE(A11, Ycol, X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
Y <- matrix(cbind(Y,Ysol), nrow = rows.A12, ncol= i)
}
#print("Y")
#print(Y)

#2) Calcular el complemento de Schur del bloque A11 en A de la siguiente forma
# Calculamos S
S=A22-(A21%*%Y)
#print("S:")
#print(S)
    
# Calculamos b_hat
b_hat=b2-(A21%*%y)
#print("b_hat")
#print(b_hat)
    
#3) Obtenemos x2 resolviendo Sx2=b_hat
### Con el método de Jacobi o GaussSeidel
x2 <- funcResolverSE(S, b_hat, X02, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
#print("x2:")
#print(x2)

#4) Obtenemos x1 resolviendo A11x1=b1-(A12*x2)
###Con el método de Jacobi o GaussSeidel
x1 <- funcResolverSE(A11, b1-(A12%*%x2) , X01, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
#print("x1:")
#print(x1)
    
c(x1,x2)
    
}

funcEliminacionBloques <- function(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo){
  # Aquellos procedimientos que utilicen SEL utilizarán los métodos de Jacobi o GaussSeidel.
  # El sistema de ecuaciones sólo se procesará si pasa
  # todas las validaciones requeridas.
  #
  # Parámetros
  # -------
  # mtrx_A : matriz
  #    Matriz a evaluar
  # vct_B : vector
  #    Vector de resultados del sistema de ecuaciones
  # vct_X0 : vector
  #    Vector de aproximaciones
  # nbr_MaxIteraciones : número
  #    Máximo número de iteraciones a alcanzar dentro del método SEL (Jacobi o GaussSiedel)
  # nbr_Threshold : número
  #    Diferencia mínima a la que se quiere llegar entre iteraciones para
  #    considerar que el método ha convergido dentro del método SEL (Jacobi o GaussSiedel)
  # str_Metodo : cadena
  #    Cadena mediante la cual se especifica el método que se empleará para
  #    actualizar el vector de aproximaciones dentro del método SEL (Jacobi o GaussSiedel)
  #
  # Regresa
  # -------
  # vector
  #    x1 como la solución del sistema

  # Condición para validar que la matriz no sea singular

  # Se inicializa el vector de resultados
  #vct_XRslt <- rep(NA, size(vct_X0)[2])

  if(!is.singular.matrix(mtrx_A)){

    if (str_Metodo == 'J' || str_Metodo == 'GS'){

      if (str_Metodo=='J'){
        #print('Solucion mediante metodo de Jacobi')
      }
      if (str_Metodo=='GS'){
        #print('Solucion mediante metodo de Gauss-Seidel')
      }

      #print('Matriz A:')
      #print(mtrx_A)

      #print('Vector b:')
      #print(vct_B)

      # Se aplican las validaciones de manera anidada
      if (funcEsVectorValido(mtrx_A, vct_B)){

        if (funcEsVectorValido(mtrx_A, vct_X0)){

          if (funcEsMatrizCuadrada(mtrx_A) == TRUE){

            # Obtenemos la n de la matriz
            n <- nrow(mtrx_A)

            if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

              # Se manda a llamar la función de eliminación por bloques
              x1_res <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

              # Se imprime el resultado
              print('Resultado final (x1) para eliminación por bloques: ')
              print(x1_res)

            } else {
              print('La matriz tiene algun cero en la diagonal, comienza ordenamiento')

              # Se ordena la matriz
              lt_Obj <- funcOrdenarEcuaciones(mtrx_A, vct_B)
              mtrx_A <- lt_Obj$matriz
              vct_B <- lt_Obj$vector

              #print('Matriz ordenada:')
              #print(mtrx_A)

              #print('Vector ordenado:')
              #print(vct_B)

              if (funcHayCeroEnDiagonal(mtrx_A) == FALSE){

                # Se manda a llamar la función del método de eliminación por bloques
                x1_res <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

                # Se imprime el resultado
                print('Resultado final (x1) para eliminación por bloques: ')
                print(x1_res)

                # En caso de encontrar algún problema, se imprime 
              } else {
                print('Pese al reordenamiento, aun hay ceros en la diagonal')
                }
            }

          } else {
              print('La matriz no cumple con ser de dimensiones nxn')
            }
            
        } else {
          print('El vector de aproximaciones no es de las dimensiones esperadas')
          }
          
      } else {
        print('El vector de resultados no es de las dimensiones esperadas')
        }

    } else{
      print("El metodo especificado no es valido, se espera 'GS' para Gauss-Seidel o 'J' para Jacobi")
      }
      
  } else{
    print('La matriz no puede ser singular')
    }

  #Se devuelve x1
  #print("Resultado Final")
  #print(x1_res)

}


In [9]:
################### 4.- Declaración de variables y pruebas finales ##########################

# Construcción de los objetos matemáticos

# Ejemplo de Erick (en desorden)
# Matriz A
# mtrx_A <- matrix(c( 0,3,-1,8,
#                    -1,11,-1,3,
#                     2,-1,10,-1,
#                    10,-1,2,0),
#                 byrow = TRUE,
#                 nrow=4,
#                 ncol=4)

# vct_B <- c(15, 25, -11, 6)
# vct_B <- c(6, 25, -11, 15)

# Ejemplo que puso Ana en el chat
# Matriz A
mtrx_A <- matrix(c(2,0,1,1,
                   0,1,0,1,
                   1,2,0,2,
                   1,1,1,1),
                   nrow=4,
                   ncol=4,
                  byrow = TRUE)

# Vector b
vct_B <- c(10, 24, 31, 45)

# Vector de aproximación inicial
vct_X0 <- c(0,0,0,0)

# Máximo número de iteraciones permitido
nbr_MaxIteraciones <- 100

# Threshold que se busca alcanzar
nbr_Threshold <- 10**(-3)

# Método a utilizar
# str_Metodo <- 'J'
str_Metodo <- 'GS'

##################### Pruebas del método de Jacobi o Gauss-Seidel ##########################

# Se manda a llamar la función que contiene las validaciones y ejecución
# del método de Jacobi o Gauss-Seide, según se indique
# vct_Solucion <- funcResolverSE(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

##################### Pruebas del método de eliminación por bloques #######################

# Se manda a llamar la función que contiene las validaciones y ejecución
# del método de Jacobi o Gauss-Seidel, según se indique#x1_Solucion <- EliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)
x1_Solucion <- funcEliminacionBloques(mtrx_A, vct_B, vct_X0, nbr_MaxIteraciones, nbr_Threshold, str_Metodo)

[1] "La matriz tiene algun cero en la diagonal, comienza ordenamiento"
[1] "Soluci\303\263n mediante metodo de Gauss-Seidel"
[1] "Matriz A:"
     [,1] [,2]
[1,]    2    0
[2,]    1    2
[1] "Vector b:"
[1] 10 31
[1] "Iteracion 0"
[1] 0 0
[1] "Iteracion 1"
[1]  5 13
[1] "nbr_Numerador: 13"
[1] "nbr_Denominador: 13"
[1] "nbr_Diff: 1"
[1] "Iteracion 2"
[1]  5 13
[1] "nbr_Numerador: 0"
[1] "nbr_Denominador: 13"
[1] "nbr_Diff: 0"
[1] "Se alcanza el threshold"
[1] "Resultado final: "
[1]  5 13
[1] "Soluci\303\263n mediante metodo de Gauss-Seidel"
[1] "Matriz A:"
     [,1] [,2]
[1,]    2    0
[2,]    1    2
[1] "Vector b:"
[1] 1 0
[1] "Iteracion 0"
[1] 0 0
[1] "Iteracion 1"
[1]  0.50 -0.25
[1] "nbr_Numerador: 0.5"
[1] "nbr_Denominador: 0.5"
[1] "nbr_Diff: 1"
[1] "Iteracion 2"
[1]  0.50 -0.25
[1] "nbr_Numerador: 0"
[1] "nbr_Denominador: 0.5"
[1] "nbr_Diff: 0"
[1] "Se alcanza el threshold"
[1] "Resultado final: "
[1]  0.50 -0.25
[1] "Soluci\303\263n mediante metodo de Gauss-Seidel"
[1] "Matriz 